# Skill 2 : Data Analysis from CSV file

To really have a Smart Search Engine or Virtual assistant that can answer any question about your corporate documents, this "engine" must understand tabular data, aka, sources with tables, rows and columns with numbers. 
This is a different problem that simply looking for the top most similar results.  The concept of indexing, bringing top results, embedding, doing a cosine semantic search and summarize an answer, doesn't really apply to this problem.
We are dealing now with sources with Tables in which each row and column are related to each other, and in order to answer a question, all of the data is needed, not just top results.

In this notebook, the goal is to show how to deal with this kind of use cases. To continue with our Covid-19 theme, we will be using an open dataset called ["Covid Tracking Project"](https://covidtracking.com/data/download). The COVID Tracking Project dataset is a  CSV file that provides the latest numbers on tests, confirmed cases, hospitalizations, and patient outcomes from every US state and territory (they stopped tracking on March 7 2021).

Imagine that many documents on a data lake are tabular data, or that your use case is to ask questions in natural language to a LLM model and this model needs to get the context from a CSV file or even a SQL Database in order to answer the question. A GPT Smart Search Engine, must understand how to deal with this sources, understand the data and answer acoordingly.

In [1]:
import os
import pandas as pd
from langchain_openai import AzureChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent, create_pandas_dataframe_agent

from common.prompts import CSV_PROMPT_PREFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

## Download the dataset and load it into Pandas Dataframe

In [3]:
os.makedirs("data",exist_ok=True)

In [4]:
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/

--2024-07-10 04:28:44--  https://covidtracking.com/data/download/all-states-history.csv
Resolving covidtracking.com (covidtracking.com)... 172.67.183.132, 104.21.64.114, 2606:4700:3034::6815:4072, ...
Connecting to covidtracking.com (covidtracking.com)|172.67.183.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./data/all-states-history.csv.3’

all-states-history.     [ <=>                ]   2.61M  --.-KB/s    in 0.03s   

2024-07-10 04:28:44 (93.4 MB/s) - ‘./data/all-states-history.csv.3’ saved [2738601]



In [5]:
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
print("Rows and Columns:",df.shape)
df.head()

Rows and Columns: (20780, 41)


,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-07,AK,305.0,0.0,0,0.0,1293.0,1293.0,33.0,0,...,1731628.0,0,0.0,0.0,0.0,0.0,0.0,0,1731628.0,0
1,2021-03-07,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,...,2323788.0,2347,0.0,0.0,119757.0,0.0,2323788.0,2347,0.0,0
2,2021-03-07,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,...,2736442.0,3380,0.0,0.0,0.0,481311.0,0.0,0,2736442.0,3380
3,2021-03-07,AS,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,2140.0,0,0.0,0.0,0.0,0.0,0.0,0,2140.0,0
4,2021-03-07,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,...,7908105.0,45110,580569.0,0.0,444089.0,0.0,3842945.0,14856,7908105.0,45110


In [6]:
df.columns

Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'recovered', 'totalTestEncountersViral',
       'totalTestEncountersViralIncrease', 'totalTestResults',
       'totalTestResultsIncrease', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalTestsPeopleViral', 'totalTestsPeopleViralIncrease',
       'totalTestsViral', 'totalTestsViralIncrease'

## CSV/Pandas Agent

LLMs are great for building question-answering systems over various types of data sources. In this section we’ll go over how to build Q&A systems over data stored in a CSV file(s). Like we did in the last notebook, the key to working with tabular CSV files is to give an LLM access to tools/experts for querying and interacting with the data

In [7]:
# Let's delve into a challenging question that demands a multi-step solution. 
# The path to solving it might not be immediately clear.
# When examining the dataframe above, even a human might struggle to determine which columns are pertinent.

QUESTION = """
How may patients were hospitalized during July 2020 in Texas. 
And nationwide as the total of all states? 
Use the hospitalizedIncrease column
"""

In [8]:
# First we load our LLM
COMPLETION_TOKENS = 1000
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

Now we need our agent and our expert/tool.  
LangChain has created an out-of-the-box agents that we can use to solve our Q&A to CSV tabular data file problem.

In [10]:
agent_executor = create_pandas_dataframe_agent(llm=llm,
                                               df=df,
                                               verbose=True,
                                               agent_type="openai-tools",
                                               allow_dangerous_code=True)

In [11]:
%%time
try:
    display(Markdown(agent_executor.invoke(QUESTION)['output']))
except Exception as e:
    print(e)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['state'] == 'TX') & (df['date'] >= '2020-07-01') & (df['date'] <= '2020-07-31')]['hospitalizedIncrease'].sum()"}`


0
Invoking: `python_repl_ast` with `{'query': "df[(df['date'] >= '2020-07-01') & (df['date'] <= '2020-07-31')]['hospitalizedIncrease'].sum()"}`


63105During July 2020, there were 0 patients hospitalized in Texas according to the `hospitalizedIncrease` column. 

Nationwide, the total number of patients hospitalized during the same period was 63,105.

> Finished chain.


During July 2020, there were 0 patients hospitalized in Texas according to the `hospitalizedIncrease` column. 

Nationwide, the total number of patients hospitalized during the same period was 63,105.

CPU times: user 188 ms, sys: 8.42 ms, total: 196 ms
Wall time: 7.97 s


That was good pandas code, and pretty fast. Let's add some prompt engineering to improve answer.

### Improving the Prompt

We can give it now an improved sager prompt. In `prompts.py` you can find `CSV_PROMPT_PREFIX`

In [12]:
CSV_PROMPT_PREFIX

'\n- First set the pandas display options to show all the columns, get the column names, then answer the question.\n- **ALWAYS** before giving the Final Answer, try another method. Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question. If you are not sure, try another method.\n- If the methods tried do not give the same result, reflect and try again until you have two methods that have the same result. \n- If you still cannot arrive to a consistent result, say that you are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful and thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**. \n- **ALWAYS**, as part of your "Final Answer", explain how you got to the answer on a section that starts with: "\n\nExplanation:\n". In the explanation, mention the column names that you used to get to the final answer. \

In [13]:
agent_executor = create_pandas_dataframe_agent(llm=llm,
                                               df=df,
                                               prefix=CSV_PROMPT_PREFIX,
                                               verbose=True,
                                               agent_type="openai-tools",
                                               allow_dangerous_code=True)

In [14]:
%%time
try:
    display(Markdown(agent_executor.invoke(QUESTION)['output']))
except Exception as e:
    print(e)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\npd.set_option('display.max_columns', None)"}`



Invoking: `python_repl_ast` with `{'query': 'df.columns'}`


Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'recovered', 'totalTestEncountersViral',
       'totalTestEncountersViralIncrease', 'totalTestResults',
       'totalTestResultsIncrease', 'tot

### Number of Patients Hospitalized in Texas and Nationwide during July 2020

During the month of July 2020, the number of patients hospitalized in Texas and nationwide can be determined using the `hospitalizedIncrease` column from the dataset.

#### Texas
After filtering the data for Texas (`state == 'TX'`) and the date range for July 2020, the sum of the `hospitalizedIncrease` column is 0. This means that there were no new hospitalizations reported in Texas during this period according to the given data.

#### Nationwide
For the nationwide data, after filtering for the date range of July 2020, the sum of the `hospitalizedIncrease` column is 63,105. This represents the total number of new hospitalizations across all states during July 2020.

### Final Answer:
- **Texas (July 2020):** 0 new hospitalizations
- **Nationwide (July 2020):** 63,105 new hospitalizations

#### Explanation:
To arrive at these results, I performed the following steps:
1. **Set display options:** Ensured all columns are visible.
2. **Filtered Data:**
   - For Texas, I filtered the dataframe for rows where `state == 'TX'` and the date range was between `2020-07-01` and `2020-07-31`.
   - For nationwide data, I filtered the dataframe for rows where the date range was between `2020-07-01` and `2020-07-31`.
3. **Summed the `hospitalizedIncrease` column:** 
   - For Texas, the sum was 0.
   - For nationwide, the sum was 63,105.

Both methods consistently showed that Texas had no new hospitalizations reported for July 2020, while the nationwide total was 63,105.

CPU times: user 1.07 s, sys: 10.4 ms, total: 1.08 s
Wall time: 34.3 s


## Evaluation
Let's see if the answer is correct

In [15]:
#df['date'] = pd.to_datetime(df['date'])
july_2020 = df[(df['date'] >= '2020-07-01') & (df['date'] <= '2020-07-31')]
texas_hospitalized_july_2020 = july_2020[july_2020['state'] == 'TX']['hospitalizedIncrease'].sum()
nationwide_hospitalized_july_2020 = july_2020['hospitalizedIncrease'].sum()

In [16]:
print( "TX:",texas_hospitalized_july_2020,"Nationwide:",nationwide_hospitalized_july_2020)

TX: 0 Nationwide: 63105


It is Correct!

**Note**: Obviously, there were hospitalizations in Texas in July 2020 (Try asking ChatGPT), but this particular File, for some reason has 0 on the column "hospitalizedIncrease" for Texas in July 2020. This proves though that the model is NOT making up information or using prior knowledge, but instead using only the results of its calculation on this CSV file. That's what we need!

# Summary

So, we just solved our problem on how to ask questions in natural language to our Tabular data hosted on a CSV File.
With this approach you can see then that it is NOT necessary to make a dump of a database data into a CSV file and index that on a Search Engine, you don't even need to use the above approach and deal with a CSV data dump file. With the Agents framework, the best engineering decision is to interact directly with the data source API without the need to replicate the data in order to ask questions to it. Remember, LLMs can do SQL very well. 


**Note**: We don't recommend using a pandas agent to answer questions from tabular data. It is not fast and it makes too many parsing mistakes. We recommend using SQL (see next notebook).

# NEXT
We can see that GPT-3.5, and even better GPT-4, is powerful and can translate a natural language question into the right steps in python in order to query a CSV data loaded into a pandas dataframe. 
That's pretty amazing. However the question remains: **Do I need then to dump all the data from my original sources (Databases, ERP Systems, CRM Systems) in order to be searchable by a Smart Search Engine?**

The next Notebook answers this question by implementing a Question->SQL process and get the information from data in a SQL Database, eliminating the need to dump it out.